In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

In [2]:
# Read data
df = pd.read_csv('data/winemag-data-130k-v2.csv')
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


## Data Inspection

In [3]:
df.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

In [4]:
df.shape

(129971, 14)

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,129971.0,64985.000000,37519.540256,0.0,32492.5,64985.0,97477.5,129970.0
points,129971.0,88.447138,3.039730,80.0,86.0,88.0,91.0,100.0
price,120975.0,35.363389,41.022218,4.0,17.0,25.0,42.0,3300.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            92506 non-null   object 
 4   points                 129971 non-null  int64  
 5   price                  120975 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               108724 non-null  object 
 8   region_2               50511 non-null   object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                129970 non-null  object 
 13  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(11)


In [7]:
df.dtypes

Unnamed: 0                 int64
country                   object
description               object
designation               object
points                     int64
price                    float64
province                  object
region_1                  object
region_2                  object
taster_name               object
taster_twitter_handle     object
title                     object
variety                   object
winery                    object
dtype: object

In [8]:
df.isna().sum()

Unnamed: 0                   0
country                     63
description                  0
designation              37465
points                       0
price                     8996
province                    63
region_1                 21247
region_2                 79460
taster_name              26244
taster_twitter_handle    31213
title                        0
variety                      1
winery                       0
dtype: int64

## Data Preprocessing

In [9]:
# Data value changes->cleaning dirty data
# 1. Drop Column : Unnamed(index column), region_2(too many null data)
# 2. Drop sample : contry, province

In [10]:
df=df.drop(columns=['Unnamed: 0','region_2'])
df= df.dropna(subset=['country', 'province','variety'])
df

,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...
129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [11]:
# 필요한 속성 : points, taster_name, title
# 필요한 속성만 추출, 속성에서 빈 sample 값 버리기
data=df[['points', 'taster_name', 'title']]
data=data.dropna(subset=['points', 'taster_name', 'title'])
data

,points,taster_name,title
0,87,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna)
1,87,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro)
2,87,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley)
3,87,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...
4,87,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...
...,...,...,...
129966,90,Anna Lee C. Iijima,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...
129967,90,Paul Gregutt,Citation 2004 Pinot Noir (Oregon)
129968,90,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...
129969,90,Roger Voss,Domaine Marcel Deiss 2012 Pinot Gris (Alsace)


In [12]:
# taster_name 컬럼의 고유한 값 확인
unique_tasters = data['taster_name'].unique()
print(unique_tasters)

['Kerin O’Keefe' 'Roger Voss' 'Paul Gregutt' 'Alexander Peartree'
 'Michael Schachner' 'Anna Lee C. Iijima' 'Virginie Boone' 'Matt Kettmann'
 'Sean P. Sullivan' 'Jim Gordon' 'Joe Czerwinski' 'Anne Krebiehl\xa0MW'
 'Lauren Buzzeo' 'Mike DeSimone' 'Jeff Jenssen' 'Susan Kostrzewa'
 'Carrie Dykes' 'Fiona Adams' 'Christina Pickard']


In [13]:
# taster_name을 기준으로 정렬
data = data.sort_values(by='taster_name')
data

,points,taster_name,title
94030,85,Alexander Peartree,Sutcliffe 2013 Chardonnay (Colorado)
740,85,Alexander Peartree,McPherson 2014 Sangiovese (Texas)
87950,84,Alexander Peartree,Illinois Sparkling Co. NV Heirloom Sparkling R...
87954,84,Alexander Peartree,Oliver 2012 Dry Red (America)
87955,84,Alexander Peartree,Pearmund 2013 Old Vine Meriwether Vineyard Cha...
...,...,...,...
32151,86,Virginie Boone,Borra 2010 White Fusion White (Lodi)
86190,93,Virginie Boone,Hunt & Ryde 2013 Trophy Red (Rockpile)
68955,89,Virginie Boone,VGS 2014 Potelle Two Red (Napa Valley)
120199,92,Virginie Boone,Quivira 2013 Black Boar Zinfandel (Dry Creek V...


In [14]:
# 너무 많은 사람에 대한 리뷰가 있기 때문에, 먼저 사람을 선별하기 위해 User-Based CF를 사용, 선별된 인원 간에 Matrix Factorization으로 최종 추천한다.
# User based collaborative filtering

In [15]:
# 피벗 테이블 생성 (행: 사용자, 열: 와인 타이틀, 값: 평점)
user_item_matrix = data.pivot_table(index='taster_name', columns='title', values='points')

# 결측값을 0으로 채움 (평가하지 않은 아이템을 0으로 처리)
user_item_matrix = user_item_matrix.fillna(0)

# 사용자 간 유사도 계산 (코사인 유사도)
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
user_similarity_df

taster_name,Alexander Peartree,Anna Lee C. Iijima,Anne Krebiehl MW,Carrie Dykes,Christina Pickard,Fiona Adams,Jeff Jenssen,Jim Gordon,Joe Czerwinski,Kerin O’Keefe,Lauren Buzzeo,Matt Kettmann,Michael Schachner,Mike DeSimone,Paul Gregutt,Roger Voss,Sean P. Sullivan,Susan Kostrzewa,Virginie Boone
taster_name,,,,,,,,,,,,,,,,,,,
Alexander Peartree,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
Anna Lee C. Iijima,0.0,1.000000,0.000000,0.001283,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000456,0.000000
Anne Krebiehl MW,0.0,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000235,0.0,0.0,0.0,0.0,0.0,0.000000,0.001692,0.000000,0.000000,0.000000
Carrie Dykes,0.0,0.001283,0.000000,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
Christina Pickard,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.006580,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.004396
Fiona Adams,0.0,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
Jeff Jenssen,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
Jim Gordon,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.002077
Joe Czerwinski,0.0,0.000000,0.000235,0.000000,0.006580,0.0,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000277,0.000000,0.000000,0.000000


In [16]:
# 특정 사용자에 대한 추천 함수
def recommend_wines(user_name, num_recommendations=5):
    # 선택한 사용자와 다른 사용자 간의 유사도 가져오기
    similar_users = user_similarity_df[user_name].sort_values(ascending=False)
    
    # 유사한 사용자가 높게 평가한 아이템 가져오기
    recommendations = pd.Series(dtype='float64')
    for similar_user, score in similar_users.iloc[1:].items():  # 자신 제외
        similar_user_ratings = user_item_matrix.loc[similar_user]
        similar_user_recommendations = similar_user_ratings[similar_user_ratings > 0] * score
        recommendations = pd.concat([recommendations, similar_user_recommendations])
    
    # 동일 아이템의 추천 점수를 합산
    recommendations = recommendations.groupby(recommendations.index).sum()
    
    # 이미 평가한 아이템은 제외하고 추천
    user_rated_items = user_item_matrix.loc[user_name][user_item_matrix.loc[user_name] > 0].index
    recommendations = recommendations[~recommendations.index.isin(user_rated_items)]
    
    # 평점이 높은 상위 아이템 추천
    recommendations = recommendations.sort_values(ascending=False).head(num_recommendations)
    return recommendations.index.tolist()

In [17]:
# 예시: 특정 사용자에게 와인 추천
user_name = 'Alexander Peartree'
recommended_wines = recommend_wines(user_name, num_recommendations=5)
print(f"추천 와인 (사용자 {user_name}):")
for i in recommended_wines:
    print(i)

추천 와인 (사용자 Alexander Peartree):
1+1=3 2008 Rosé Cabernet Sauvignon (Penedès)
Moët & Chandon 2005 Dom Pérignon Brut Rosé  (Champagne)
Moët & Chandon 2004 Grand Vintage Brut Rosé  (Champagne)
Moët & Chandon 2004 Dom Pérignon Brut Rosé  (Champagne)
Moët & Chandon 2003 Vintage Brut  (Champagne)


In [18]:
unique_tasters = data['taster_name'].unique()
for tester in unique_tasters:
    user_name=tester
    recommended_wines = recommend_wines(user_name, num_recommendations=5)
    print(f"추천 와인 (사용자 {user_name}):")
    for wine in recommended_wines:
        print(wine)
    print()

추천 와인 (사용자 Alexander Peartree):
1+1=3 2008 Rosé Cabernet Sauvignon (Penedès)
Moët & Chandon 2005 Dom Pérignon Brut Rosé  (Champagne)
Moët & Chandon 2004 Grand Vintage Brut Rosé  (Champagne)
Moët & Chandon 2004 Dom Pérignon Brut Rosé  (Champagne)
Moët & Chandon 2003 Vintage Brut  (Champagne)

추천 와인 (사용자 Anna Lee C. Iijima):
King Family 2015 Orange Viognier (Monticello)
Barboursville Vineyards 2015 Reserve Cabernet Franc (Virginia)
The Williamsburg Winery 2015 Adagio Red (Virginia)
Lovingston 2015 Josie's Knoll Rotunda Red (Monticello)
Fabbioli Cellars 2012 Reserve Cabernet Franc (Virginia)

추천 와인 (사용자 Anne Krebiehl MW):
Laurent-Perrier NV Cuvée Brut Rosé  (Champagne)
Ruinart NV Brut Rosé  (Champagne)
Quinta do Noval 2011 Nacional Vintage  (Port)
Château Cheval Blanc 2010  Saint-Émilion
Krug 2002 Brut  (Champagne)

추천 와인 (사용자 Carrie Dykes):
Robert Weil 2015 Kiedrich Gräfenberg Trockenbeerenauslese Riesling (Rheingau)
Robert Weil 2014 Kiedrich Gräfenberg Trockenbeerenauslese Riesling (Rhe

In [19]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import numpy as np

# 피벗 테이블 생성 (행: 사용자, 열: 와인 타이틀, 값: 평점)
user_item_matrix = data.pivot_table(index='taster_name', columns='title', values='points')

# 결측값을 0으로 채움 (평가하지 않은 아이템을 0으로 처리)
user_item_matrix = user_item_matrix.fillna(0)

# 사용자 간 유사도 계산 (코사인 유사도)
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# 특정 사용자의 유사도가 높은 상위 N명의 사용자 선택
def get_similar_users(user_name, top_n=10):
    similar_users = user_similarity_df[user_name].sort_values(ascending=False).iloc[1:top_n+1]
    return similar_users.index.tolist()

# Matrix Factorization을 통한 추천
def matrix_factorization_recommendation(user_name, num_recommendations=5, top_n_similar_users=10):
    # 유사도가 높은 사용자를 선택
    similar_users = get_similar_users(user_name, top_n=top_n_similar_users)
    
    # 유사한 사용자만을 포함하는 부분 매트릭스 생성
    similar_users_matrix = user_item_matrix.loc[similar_users]
    
    # Truncated SVD를 사용하여 Matrix Factorization 수행
    svd = TruncatedSVD(n_components=20)
    user_item_matrix_reduced = svd.fit_transform(similar_users_matrix)
    reconstructed_matrix = np.dot(user_item_matrix_reduced, svd.components_)
    reconstructed_df = pd.DataFrame(reconstructed_matrix, index=similar_users, columns=user_item_matrix.columns)
    
    # 선택한 사용자가 평가하지 않은 아이템에 대해 예측 평점 선택
    user_unrated_items = user_item_matrix.loc[user_name][user_item_matrix.loc[user_name] == 0].index
    predicted_ratings = reconstructed_df.loc[similar_users].mean(axis=0)[user_unrated_items]
    
    # 예측 평점이 높은 아이템을 추천
    recommendations = predicted_ratings.sort_values(ascending=False).head(num_recommendations)
    return recommendations.index.tolist()


In [20]:
unique_tasters = data['taster_name'].unique()
for tester in unique_tasters:
    user_name=tester
    final_recommended_wines = matrix_factorization_recommendation(user_name, num_recommendations=5)
    print(f"추천 와인 (사용자 {user_name}):")
    for wine in recommended_wines:
        print(wine)
    print()

추천 와인 (사용자 Alexander Peartree):
Torbreck 2006 The Struie Shiraz (Barossa)
Domaine Chandon 2000 Pinot Meunier (Carneros)
MacMurray Ranch 2011 Pinot Noir (Russian River Valley)
Domaine Chandon 2000 Chardonnay (Carneros)
Keating 2012 Sauvignon Blanc (Dry Creek Valley)

추천 와인 (사용자 Anna Lee C. Iijima):
Torbreck 2006 The Struie Shiraz (Barossa)
Domaine Chandon 2000 Pinot Meunier (Carneros)
MacMurray Ranch 2011 Pinot Noir (Russian River Valley)
Domaine Chandon 2000 Chardonnay (Carneros)
Keating 2012 Sauvignon Blanc (Dry Creek Valley)

추천 와인 (사용자 Anne Krebiehl MW):
Torbreck 2006 The Struie Shiraz (Barossa)
Domaine Chandon 2000 Pinot Meunier (Carneros)
MacMurray Ranch 2011 Pinot Noir (Russian River Valley)
Domaine Chandon 2000 Chardonnay (Carneros)
Keating 2012 Sauvignon Blanc (Dry Creek Valley)

추천 와인 (사용자 Carrie Dykes):
Torbreck 2006 The Struie Shiraz (Barossa)
Domaine Chandon 2000 Pinot Meunier (Carneros)
MacMurray Ranch 2011 Pinot Noir (Russian River Valley)
Domaine Chandon 2000 Chardonnay 

In [21]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from implicit.als import AlternatingLeastSquares
import scipy.sparse as sparse
import numpy as np

# 피벗 테이블 생성 (행: 사용자, 열: 와인 타이틀, 값: 평점)
user_item_matrix = data.pivot_table(index='taster_name', columns='title', values='points')

# 결측값을 0으로 채움 (평가하지 않은 아이템을 0으로 처리)
user_item_matrix = user_item_matrix.fillna(0)

# 사용자 간 유사도 계산 (코사인 유사도)
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# 특정 사용자의 유사도가 높은 상위 N명의 사용자 선택
def get_similar_users(user_name, top_n=10):
    similar_users = user_similarity_df[user_name].sort_values(ascending=False).iloc[1:top_n+1]
    return similar_users.index.tolist()

# ALS를 통한 추천
def als_recommendation(user_name, num_recommendations=5, top_n_similar_users=10):
    # 유사도가 높은 사용자를 선택
    similar_users = get_similar_users(user_name, top_n=top_n_similar_users)
    
    # 유사한 사용자만을 포함하는 부분 매트릭스 생성
    similar_users_matrix = user_item_matrix.loc[similar_users]
    
    # ALS 모델을 위한 희소 행렬 생성
    sparse_user_item = sparse.csr_matrix(similar_users_matrix.values)
    
    # ALS 모델 생성 및 학습
    als_model = AlternatingLeastSquares(factors=20, iterations=15, regularization=0.1)
    als_model.fit(sparse_user_item.T)  # ALS 모델은 아이템-사용자 형식으로 학습
    
    # 선택한 사용자의 인덱스
    user_index = list(similar_users_matrix.index).index(user_name)
    
    # 추천 생성
    user_recommendations = als_model.recommend(
        user_index, sparse_user_item[user_index], N=num_recommendations, filter_already_liked_items=True
    )
    
    # 추천 아이템의 인덱스를 원래 와인 타이틀로 변환
    recommended_items = [similar_users_matrix.columns[i] for i, _ in user_recommendations]
    return recommended_items


In [22]:
unique_tasters = data['taster_name'].unique()
for tester in unique_tasters:
    user_name=tester
    final_recommended_wines = matrix_factorization_recommendation(user_name, num_recommendations=5)
    print(f"추천 와인 (사용자 {user_name}):")
    for wine in recommended_wines:
        print(wine)
    print()

추천 와인 (사용자 Alexander Peartree):
Torbreck 2006 The Struie Shiraz (Barossa)
Domaine Chandon 2000 Pinot Meunier (Carneros)
MacMurray Ranch 2011 Pinot Noir (Russian River Valley)
Domaine Chandon 2000 Chardonnay (Carneros)
Keating 2012 Sauvignon Blanc (Dry Creek Valley)

추천 와인 (사용자 Anna Lee C. Iijima):
Torbreck 2006 The Struie Shiraz (Barossa)
Domaine Chandon 2000 Pinot Meunier (Carneros)
MacMurray Ranch 2011 Pinot Noir (Russian River Valley)
Domaine Chandon 2000 Chardonnay (Carneros)
Keating 2012 Sauvignon Blanc (Dry Creek Valley)

추천 와인 (사용자 Anne Krebiehl MW):
Torbreck 2006 The Struie Shiraz (Barossa)
Domaine Chandon 2000 Pinot Meunier (Carneros)
MacMurray Ranch 2011 Pinot Noir (Russian River Valley)
Domaine Chandon 2000 Chardonnay (Carneros)
Keating 2012 Sauvignon Blanc (Dry Creek Valley)

추천 와인 (사용자 Carrie Dykes):
Torbreck 2006 The Struie Shiraz (Barossa)
Domaine Chandon 2000 Pinot Meunier (Carneros)
MacMurray Ranch 2011 Pinot Noir (Russian River Valley)
Domaine Chandon 2000 Chardonnay 